In [5]:
import numpy as np
import os
import tensorflow as tf
# slim = tf.contrib.slim
# FLAGS = tf.app.flags.FLAGS
# import urllib2
# import sys
# import matplotlib.pyplot as plt
# import sklearn.metrics as sm

# sys.path.append('/home/jt2/Workspace/models/slim')
# from datasets import dataset_utils
# from nets import inception
# from preprocessing import inception_preprocessing
# import image_processing
from __future__ import division
import cervix
%aimport utils
%aimport train
from utils import *
from train import *

In [3]:
set_flag('model', 'inception_v1')
# set_flag('cervix_data_dir', '/home/jt2/Workspace/cervix/train/')
# set_flag('train_dir', '/tmp/inception_finetuned/'
set_flag('image_size', inception.inception_v1.default_image_size)
set_flag('data_dir', '/home/jt2/Workspace/cervix/30_img/')
set_flag('log_dir', '/tmp/inception_finetuned/30_img')
set_flag('checkpoints_dir', '/tmp/checkpoints')
set_flag('learning_rate', 0.001)
set_flag('number_of_steps', 2)

if FLAGS.model == 'inception_v1':
    set_flag('url', 'http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz')

tf.logging.set_verbosity(tf.logging.INFO)

In [9]:
Train(cervix).init_train()

Instructions for updating:
Use tf.losses.softmax_cross_entropy instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Instructions for updating:
Use tf.losses.get_total_loss instead.
Instructions for updating:
Use tf.losses.get_losses instead.
Instructions for updating:
Use tf.losses.get_regularization_losses instead.
INFO:tensorflow:Summary name losses/Total Loss is illegal; using losses/Total_Loss instead.
InceptionV1/Conv2d_1a_7x7/weights:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/beta:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/moving_mean:0
InceptionV1/Conv2d_1a_7x7/BatchNorm/moving_variance:0
InceptionV1/Conv2d_2b_1x1/weights:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/beta:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/moving_mean:0
InceptionV1/Conv2d_2b_1x1/BatchNorm/moving_variance:0
InceptionV1/Conv2d_2c_3x3/weights:0
InceptionV1/Conv2d_2c_3x3/BatchNorm/beta:0
InceptionV1/Conv2d_2c_3x3/BatchNorm/moving_mean:0
Inceptio

In [ ]:
# init train
with tf.Graph().as_default():
    dataset = cervix.get_split('train', cervix_data_dir)
    images, _, labels = load_batch(dataset, height=image_size, width=image_size)
    
#     Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes=dataset.num_classes, is_training=True)
        
    labels = tf.subtract(labels, 1)
    
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    one_hot_labels = tf.cast(one_hot_labels, tf.int64)
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # Run the training:
    final_loss = slim.learning.train(
        train_op,
        logdir=log_dir,
        init_fn=get_init_fn(),
        number_of_steps=FLAGS.number_of_steps)
        
  
print('Finished training. Last batch loss %f' % final_loss)

In [ ]:
# train from checkpoint
with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = cervix.get_split('train', cervix_data_dir)
    images, _, labels, _,_ = load_batch(dataset, height=image_size, width=image_size)
    
#     Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes=dataset.num_classes, is_training=True)
        
    labels = tf.subtract(labels, 1)
    
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # Run the training:
    final_loss = slim.learning.train(
        train_op,
        logdir=train_dir,
        number_of_steps=200)
        
  
print('Finished training. Last batch loss %f' % final_loss)

In [ ]:
#EVALUATE TRAINED MODEL
image_size = inception.inception_v1.default_image_size
batch_size = 1

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
#     dataset = cervix.get_split('train', cervix_data_dir)
    dataset = cervix.get_split('train', '/home/jt2/Workspace/cervix/30_img/')
#     images, images_raw, labels = load_batch(dataset, height=image_size, width=image_size, is_training=False)
    images, images_raw, labels, labels_name, filenames = load_batch(dataset, 
                                                           batch_size=30, 
                                                           shuffle=False, 
                                                           height=image_size, 
                                                           width=image_size, 
                                                           is_training=True)
    
    labels = tf.subtract(labels, 1)
    
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes=dataset.num_classes, is_training=False)

    probabilities = tf.nn.softmax(logits)
    
    checkpoint_path = tf.train.latest_checkpoint(train_dir)
    init_fn = slim.assign_from_checkpoint_fn(
      checkpoint_path,
      slim.get_variables_to_restore())
    
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.initialize_local_variables())
            init_fn(sess)
#             np_probabilities, np_images_raw, np_labels = sess.run([probabilities, images_raw, labels])
            np_probabilities, np_images_raw, np_labels, np_images, logit_train, one_hot, label_name, filename = sess.run([probabilities, images_raw, labels, images, logits, one_hot_labels, labels_name, filenames])
    
            for i in xrange(batch_size): 
                image = np_images_raw[i, :, :, :]
                image_train = np_images[i, :, :, :]
                true_label = np_labels[i]
                predicted_label = np.argmax(np_probabilities[i, :])
                print one_hot
                print logit_train
                print np_probabilities
                print np.argmax(np_probabilities, axis=1)
                print np_labels
                print "label_name"
                print label_name
                print "filename"
                print filename
                
#                 predicted_name = dataset.labels_to_names[predicted_label]
#                 true_name = dataset.labels_to_names[true_label]
                
                plt.figure()
                plt.imshow(image.astype(np.uint8))
#                 plt.title('Ground Truth: [%s], Prediction [%s]' % (true_name, predicted_name))
                plt.title('Ground Truth: [%s], Prediction [%s]' % (true_label, predicted_label))
                plt.axis('off')
                plt.show()
            
            print(sm.classification_report(np_labels, np.argmax(np_probabilities, axis=1)))

In [ ]:
pred = np.argmax(np_probabilities,axis=1)
is_right = np.equal(np_labels, pred)
log_loss = sm.log_loss(np_labels, np_probabilities)
print "Accuracy: %f" % (np.count_nonzero(is_right)/is_right.shape[0])
print "Log loss: %f" % log_loss

In [ ]:
index_1 = np.where(np_labels == 0)
index_2 = np.where(np_labels == 1)
index_3 = np.where(np_labels == 2)
df1 = pd.DataFrame(np_probabilities[index_1], index=filename[index_1])
df1 = df1.sort_index()
df2 = pd.DataFrame(np_probabilities[index_2], index=filename[index_2])
df2 = df2.sort_index()
df3 = pd.DataFrame(np_probabilities[index_3], index=filename[index_3])
df3 = df3.sort_index()
print df1
print df2
print df3

In [ ]:
print image_train.shape
image
f, (ax1, ax2, ax3) = plt.subplots(1, 3,sharey=True)
ax1.imshow(image_train[:,:,0], cmap='gray')
ax2.imshow(image_train[:,:,1], cmap='gray')
ax3.imshow(image_train[:,:,2], cmap='gray')
plt.show()

In [ ]:
#CHECK WEIGHTS
with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
#     dataset = cervix.get_split('train', cervix_data_dir)
    images, images_raw, labels = load_batch(dataset, shuffle=True, height=image_size, width=image_size, is_training=True)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes=dataset.num_classes, is_training=False)

    checkpoint_path = tf.train.latest_checkpoint(train_dir)
    init_fn = slim.assign_from_checkpoint_fn(
      checkpoint_path,
      slim.get_variables_to_restore())
    
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.initialize_local_variables())
            init_fn(sess)
#             in_logits = slim.get_variables_by_name('InceptionV1/Logits/Conv2d_0c_1x1/weights')
#             stay_same = slim.get_variables_by_name('InceptionV1/Mixed_5c/Branch_3/Conv2d_0b_1x1/weights')
            variables = slim.get_variables()
#             l_out, ss = sess.run([in_logits, stay_same])
            all_var = sess.run(variables)
        
        print "DONE"

In [ ]:
get_variables_stats(variables, all_var)

In [ ]:
utils??
